In [1]:
import pandas as pd
import numpy as np
import wave
import os
import winsound
import pydub
import time
from pydub import AudioSegment
from pathlib import Path

In [8]:
# C = 0, C# = 1, etc.
roots = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

# 1 |-> sus 2, 2 |-> minor 3rd, 3 |-> major 3rd, 4 |-> sus 4
thirds = [1,2,3,4]

# 1 |-> flat 5th, 2 |-> perfect 5th, 3 |-> sharp 5th
fifths = [1,2,3]

# 0|-> no 7th, 1 |-> minor 7th, 2 |-> major 7th
sevenths = [0,1,2]

# 0 |-> no 9th, 1 |-> flat 9th, 2 |-> natural 9th, 3 |-> sharp 9th
ninths = [0, 1, 2, 3]

# 0 |-> no 11th, 1 |-> natural 11th, 2 |-> sharp 11th
elevenths = [0, 1, 2]

# 0 |-> no 13th, 1 |-> flat 13th, 2 |-> natural 13th
thirteenths = [0, 1, 2]

In [16]:
def chord_name(chord, shift=0):
    # Inputs: chord; an array, shift, an integer
    # the chord array has a choice of root note, and the remaining elements
    # are the choice of extensions
    
    # Outputs: a string which is the name of the chord
    
    name = ""
    
    # Next we define a some dictionaries using our chord note conventions:
    root_dict = {
    0: 'C',
    1: 'C#',
    2: 'D',
    3: 'Eb',
    4: 'E',
    5: 'F',
    6: 'F#',
    7: 'G',
    8: 'G#',
    9: 'A',
    10: 'Bb',
    11: 'B'
    }
    
    third_dict = {
        1: 'sus2',
        2: 'm',
        3: '',
        4: 'sus4'
    }
    
    fifth_dict = {
        1: ' b5',
        2: '',
        3: ' #5'
    }
    
    seventh_dict = {
        0: '',
        1: ' 7',
        2: ' maj7'
    }
    
    ninth_dict = {
        0: '',
        1: ' b9',
        2: ' 9',
        3: ' #9'
    }
    
    eleventh_dict = {
        0: '',
        1: ' 11',
        2: ' #11'
    }
    
    thirteenth_dict = {
        0: '',
        1: ' b13',
        2: ' 13'
    }
    
    chord_array_dict = {
        0: root_dict,
        1: third_dict,
        2: fifth_dict,
        3: seventh_dict,
        4: ninth_dict,
        5: eleventh_dict,
        6: thirteenth_dict
    }
    
    for i in range(7):
        if i == 0:
            name += chord_array_dict[i][chord[i]+shift]
        else:
            name += chord_array_dict[i][chord[i]]
        
    return name

There are 5184 (some other number) naive chords, and a naive chord is a 7-tuple in this setup.
Thus our data frame will be a $3456 \times 7$ matrix.

Update: I realized that minor 3rd and sharp 9 are equivalent. After a basic combinatorics exercise, I determined there were 432 chords with a minor 3rd and sharp 9 simulataneously. So the new correct number of chords is 3024.

Update: Yet again, I realized that chords with both flat 5 and sharp 11 are redundant. I will thus remove all chords that have simultaneously a flat 5 and sharp 11. There are 576 of these chords, bringing the total number of chords down to 2448.
However, I realized it's unreasonable to require all chords to have a 7th degree extension, so I have decided to include chords that don't have a 7th extension. This thus brings the total number of chords to $5184-648-864+108=3780$ chords. Here $5184$ is the size of the direct product, $648$ is the number of minor 3rd/sharp 9 redundancies, $864$ is the number of flat 5/sharp 11 redundancies, and 108 is the size of the intersection of these two redundant sets.

In [22]:
chord_data = np.zeros((15552, 7))

def cartesian_coord(*arrays):
    grid = np.meshgrid(*arrays)        
    coord_list = [entry.ravel() for entry in grid]
    points = np.vstack(coord_list).T
    points[:, [1, 0]] = points[:, [0, 1]]
    return points

chord_data = cartesian_coord(
    thirds, 
    roots, 
    fifths, 
    sevenths, 
    ninths, 
    elevenths, 
    thirteenths
)

chord_data=pd.DataFrame(chord_data, columns=['root', '3rd', '5th', '7th', '9th', '11th', '13th'])

#add a chord name column
names = [""]*15552
for index, row in chord_data.iterrows():
    names[index]=chord_name(row)
    
chord_data['name'] = names

#remove redundant chords:

#remove chords with simultaneously minor 3rd and sharp 9
chord_data=chord_data[(chord_data["3rd"]!=2) |(chord_data["9th"]!=3)]
#remove chords with simultaneously flat 5 and sharp 11
chord_data=chord_data[(chord_data["5th"]!=1) |(chord_data["11th"]!=2)]
#remove chords with sharp 5 and b13
chord_data=chord_data[(chord_data["5th"]!=3) | (chord_data["13th"]!=1)]
#remove chords with sharp 5 and sharp 11, equivalent to b5 and b13
chord_data=chord_data[(chord_data["5th"]!=3) | (chord_data["11th"]!=2)]
#remove chords with sus2 and sharp 9, equivalent to a minor 9 chord
chord_data=chord_data[(chord_data['3rd']!=1) | (chord_data['9th']!=3)]
#remove chords with sus2 and natural 9, redundant note
chord_data=chord_data[(chord_data['3rd']!=1) | (chord_data['9th']!=2)]
#remove chords with sus4 and natural 11, redundant note
chord_data=chord_data[(chord_data['3rd']!=4) | (chord_data['11th']!=1)]
#remove chords with sus4 and sharp 9, equivalent to minor 11 chord
chord_data=chord_data[(chord_data['3rd']!=4) | (chord_data['9th']!=3)]
#remove chords with sus4 and natural 9, equivalent to sus2 11
chord_data=chord_data[(chord_data['3rd']!=4) | (chord_data['9th']!=2)]

chord_data.to_csv(
    r'E:\Academics\Jupyter Notebooks\data\Chord Data\chords dataset.txt', 
    header='infer', 
    index=None, 
    sep=' ', 
    mode='a'
)

In [37]:
def keys_of_chord(chord):
    chord = np.array(chord)
    offset = np.array([0, chord[0]+2, chord[0]+5, chord[0]+9, chord[0]+0, chord[0]+4, chord[0]+7])
    keys = (chord+offset)%12
    del_inds = [i for i in range(3,7) if chord[i] == 0]
    keys=np.delete(keys, del_inds)
    return keys

In [23]:
chords = pd.read_csv(
    r'E:\Academics\Jupyter Notebooks\data\Chord Data\chords dataset.txt', 
    sep=' '
)
chords

,root,3rd,5th,7th,9th,11th,13th,name
0,0,1,1,0,0,0,0,Csus2 b5
1,0,1,1,0,0,0,1,Csus2 b5 b13
2,0,1,1,0,0,0,2,Csus2 b5 13
3,0,1,1,0,0,1,0,Csus2 b5 11
4,0,1,1,0,0,1,1,Csus2 b5 11 b13
...,...,...,...,...,...,...,...,...
6943,11,4,3,1,1,0,2,Bsus4 #5 7 b9 13
6944,11,4,3,2,0,0,0,Bsus4 #5 maj7
6945,11,4,3,2,0,0,2,Bsus4 #5 maj7 13
6946,11,4,3,2,1,0,0,Bsus4 #5 maj7 b9


In [62]:
for i, chord1 in chords.iterrows():
    chord1 = chord1[:-1]
    print(i)
    for j, chord2 in chords.iterrows():
        chord2 = chord2[:-1]
        if j > i:
            if (set(keys_of_chord(chord1)) == set(keys_of_chord(chord2))):
                print("Chord 1: ")
                print(chords.iloc[i].values)
                print("Chord 2: ")
                print(chords.iloc[j].values)
                print("Notes 1: ", set(keys_of_chord(chord1)))
                print("Notes 2: ", set(keys_of_chord(chord2)))

0
1
Chord 1: 
[0 1 1 0 0 0 1 'Cm b5 b13']
Chord 2: 
[0 1 3 0 0 2 0 'Cm #5 #11']
Notes 1:  {0, 8, 3, 6}
Notes 2:  {0, 8, 3, 6}
Chord 1: 
[0 1 1 0 0 0 1 'Cm b5 b13']
Chord 2: 
[8 2 2 1 0 0 0 'G# 7']
Notes 1:  {0, 8, 3, 6}
Notes 2:  {8, 0, 3, 6}
2
Chord 1: 
[0 1 1 0 0 0 2 'Cm b5 13']
Chord 2: 
[3 1 1 0 0 0 2 'Ebm b5 13']
Notes 1:  {0, 9, 3, 6}
Notes 2:  {0, 9, 3, 6}
Chord 1: 
[0 1 1 0 0 0 2 'Cm b5 13']
Chord 2: 
[6 1 1 0 0 0 2 'F#m b5 13']
Notes 1:  {0, 9, 3, 6}
Notes 2:  {0, 9, 3, 6}
Chord 1: 
[0 1 1 0 0 0 2 'Cm b5 13']
Chord 2: 
[9 1 1 0 0 0 2 'Am b5 13']
Notes 1:  {0, 9, 3, 6}
Notes 2:  {0, 9, 3, 6}
3
4
Chord 1: 
[0 1 1 0 0 1 1 'Cm b5 11 b13']
Chord 2: 
[5 1 2 1 1 0 0 'Fm 7 b9']
Notes 1:  {0, 3, 5, 6, 8}
Notes 2:  {0, 3, 5, 6, 8}
Chord 1: 
[0 1 1 0 0 1 1 'Cm b5 11 b13']
Chord 2: 
[8 2 2 1 0 0 2 'G# 7 13']
Notes 1:  {0, 3, 5, 6, 8}
Notes 2:  {0, 3, 5, 6, 8}
5
Chord 1: 
[0 1 1 0 0 1 2 'Cm b5 11 13']
Chord 2: 
[3 1 1 0 2 0 2 'Ebm b5 9 13']
Notes 1:  {0, 3, 5, 6, 9}
Notes 2:  {0, 3, 5, 6, 

Chord 1: 
[0 1 1 1 1 0 1 'Cm b5 7 b9 b13']
Chord 2: 
[8 2 2 1 2 1 0 'G# 7 9 11']
Notes 1:  {0, 1, 3, 6, 8, 10}
Notes 2:  {0, 1, 3, 6, 8, 10}
Chord 1: 
[0 1 1 1 1 0 1 'Cm b5 7 b9 b13']
Chord 2: 
[10 1 3 1 2 1 0 'Bbm #5 7 9 11']
Notes 1:  {0, 1, 3, 6, 8, 10}
Notes 2:  {0, 1, 3, 6, 8, 10}
26
Chord 1: 
[0 1 1 1 1 0 2 'Cm b5 7 b9 13']
Chord 2: 
[3 1 2 1 0 2 2 'Ebm 7 #11 13']
Notes 1:  {0, 1, 3, 6, 9, 10}
Notes 2:  {0, 1, 3, 6, 9, 10}
Chord 1: 
[0 1 1 1 1 0 2 'Cm b5 7 b9 13']
Chord 2: 
[6 2 2 0 3 2 2 'F# #9 #11 13']
Notes 1:  {0, 1, 3, 6, 9, 10}
Notes 2:  {0, 1, 3, 6, 9, 10}
Chord 1: 
[0 1 1 1 1 0 2 'Cm b5 7 b9 13']
Chord 2: 
[10 1 3 2 2 1 0 'Bbm #5 maj7 9 11']
Notes 1:  {0, 1, 3, 6, 9, 10}
Notes 2:  {0, 1, 3, 6, 9, 10}
27
Chord 1: 
[0 1 1 1 1 1 0 'Cm b5 7 b9 11']
Chord 2: 
[3 1 2 1 2 0 2 'Ebm 7 9 13']
Notes 1:  {0, 1, 3, 5, 6, 10}
Notes 2:  {0, 1, 3, 5, 6, 10}
Chord 1: 
[0 1 1 1 1 1 0 'Cm b5 7 b9 11']
Chord 2: 
[6 2 2 2 0 2 2 'F# maj7 #11 13']
Notes 1:  {0, 1, 3, 5, 6, 10}
Notes 2:  {0, 1, 

Chord 1: 
[0 1 1 2 0 1 0 'Cm b5 maj7 11']
Chord 2: 
[3 1 3 0 2 0 2 'Ebm #5 9 13']
Notes 1:  {0, 3, 5, 6, 11}
Notes 2:  {0, 3, 5, 6, 11}
Chord 1: 
[0 1 1 2 0 1 0 'Cm b5 maj7 11']
Chord 2: 
[11 2 2 0 1 2 0 'B b9 #11']
Notes 1:  {0, 3, 5, 6, 11}
Notes 2:  {0, 3, 5, 6, 11}
40
Chord 1: 
[0 1 1 2 0 1 1 'Cm b5 maj7 11 b13']
Chord 2: 
[3 1 3 0 2 1 2 'Ebm #5 9 11 13']
Notes 1:  {0, 3, 5, 6, 8, 11}
Notes 2:  {0, 3, 5, 6, 8, 11}
Chord 1: 
[0 1 1 2 0 1 1 'Cm b5 maj7 11 b13']
Chord 2: 
[5 1 2 1 1 2 0 'Fm 7 b9 #11']
Notes 1:  {0, 3, 5, 6, 8, 11}
Notes 2:  {0, 3, 5, 6, 8, 11}
Chord 1: 
[0 1 1 2 0 1 1 'Cm b5 maj7 11 b13']
Chord 2: 
[8 2 2 1 3 0 2 'G# 7 #9 13']
Notes 1:  {0, 3, 5, 6, 8, 11}
Notes 2:  {0, 3, 5, 6, 8, 11}
Chord 1: 
[0 1 1 2 0 1 1 'Cm b5 maj7 11 b13']
Chord 2: 
[11 2 2 0 1 2 2 'B b9 #11 13']
Notes 1:  {0, 3, 5, 6, 8, 11}
Notes 2:  {0, 3, 5, 6, 8, 11}
41
Chord 1: 
[0 1 1 2 0 1 2 'Cm b5 maj7 11 13']
Chord 2: 
[3 1 3 0 2 2 2 'Ebm #5 9 #11 13']
Notes 1:  {0, 3, 5, 6, 9, 11}
Notes 2:  {0, 3, 5

Chord 1: 
[0 1 2 0 1 1 2 'Cm b9 11 13']
Chord 2: 
[5 2 2 1 2 0 1 'F 7 9 b13']
Notes 1:  {0, 1, 3, 5, 7, 9}
Notes 2:  {0, 1, 3, 5, 7, 9}
Chord 1: 
[0 1 2 0 1 1 2 'Cm b9 11 13']
Chord 2: 
[9 2 1 1 3 0 1 'A b5 7 #9 b13']
Notes 1:  {0, 1, 3, 5, 7, 9}
Notes 2:  {0, 1, 3, 5, 7, 9}
Chord 1: 
[0 1 2 0 1 1 2 'Cm b9 11 13']
Chord 2: 
[9 2 3 1 3 2 0 'A #5 7 #9 #11']
Notes 1:  {0, 1, 3, 5, 7, 9}
Notes 2:  {0, 1, 3, 5, 7, 9}
69
70
71
Chord 1: 
[0 1 2 0 1 2 2 'Cm b9 #11 13']
Chord 2: 
[3 2 1 1 3 0 2 'Eb b5 7 #9 13']
Notes 1:  {0, 1, 3, 6, 7, 9}
Notes 2:  {0, 1, 3, 6, 7, 9}
Chord 1: 
[0 1 2 0 1 2 2 'Cm b9 #11 13']
Chord 2: 
[6 1 2 0 1 2 2 'F#m b9 #11 13']
Notes 1:  {0, 1, 3, 6, 7, 9}
Notes 2:  {0, 1, 3, 6, 7, 9}
Chord 1: 
[0 1 2 0 1 2 2 'Cm b9 #11 13']
Chord 2: 
[9 2 1 1 3 0 2 'A b5 7 #9 13']
Notes 1:  {0, 1, 3, 6, 7, 9}
Notes 2:  {0, 1, 3, 6, 7, 9}
72
73
Chord 1: 
[0 1 2 0 2 0 1 'Cm 9 b13']
Chord 2: 
[8 2 2 2 0 2 0 'G# maj7 #11']
Notes 1:  {0, 2, 3, 7, 8}
Notes 2:  {0, 2, 3, 7, 8}
74
Chord 1: 
[0 1 

Chord 1: 
[0 1 2 1 1 1 1 'Cm 7 b9 11 b13']
Chord 2: 
[3 2 2 1 2 1 2 'Eb 7 9 11 13']
Notes 1:  {0, 1, 3, 5, 7, 8, 10}
Notes 2:  {0, 1, 3, 5, 7, 8, 10}
Chord 1: 
[0 1 2 1 1 1 1 'Cm 7 b9 11 b13']
Chord 2: 
[5 1 2 1 2 1 1 'Fm 7 9 11 b13']
Notes 1:  {0, 1, 3, 5, 7, 8, 10}
Notes 2:  {0, 1, 3, 5, 7, 8, 10}
Chord 1: 
[0 1 2 1 1 1 1 'Cm 7 b9 11 b13']
Chord 2: 
[7 1 1 1 1 1 1 'Gm b5 7 b9 11 b13']
Notes 1:  {0, 1, 3, 5, 7, 8, 10}
Notes 2:  {0, 1, 3, 5, 7, 8, 10}
Chord 1: 
[0 1 2 1 1 1 1 'Cm 7 b9 11 b13']
Chord 2: 
[8 2 2 2 2 1 2 'G# maj7 9 11 13']
Notes 1:  {0, 1, 3, 5, 7, 8, 10}
Notes 2:  {0, 1, 3, 5, 7, 8, 10}
Chord 1: 
[0 1 2 1 1 1 1 'Cm 7 b9 11 b13']
Chord 2: 
[10 1 2 1 2 1 2 'Bbm 7 9 11 13']
Notes 1:  {0, 1, 3, 5, 7, 8, 10}
Notes 2:  {0, 1, 3, 5, 7, 8, 10}
95
Chord 1: 
[0 1 2 1 1 1 2 'Cm 7 b9 11 13']
Chord 2: 
[1 2 3 2 2 2 2 'C# #5 maj7 9 #11 13']
Notes 1:  {0, 1, 3, 5, 7, 9, 10}
Notes 2:  {0, 1, 3, 5, 7, 9, 10}
Chord 1: 
[0 1 2 1 1 1 2 'Cm 7 b9 11 13']
Chord 2: 
[3 2 2 1 2 2 2 'Eb 7 9 #11 1

...

629
Chord 1: 
[1 1 3 2 2 2 2 'C#m #5 maj7 9 #11 13']
Chord 2: 
[3 2 2 1 1 2 2 'Eb 7 b9 #11 13']
Notes 1:  {0, 1, 3, 4, 7, 9, 10}
Notes 2:  {0, 1, 3, 4, 7, 9, 10}
Chord 1: 
[1 1 3 2 2 2 2 'C#m #5 maj7 9 #11 13']
Chord 2: 
[10 1 1 2 2 1 2 'Bbm b5 maj7 9 11 13']
Notes 1:  {0, 1, 3, 4, 7, 9, 10}
Notes 2:  {0, 1, 3, 4, 7, 9, 10}
630
631
Chord 1: 
[1 2 1 0 0 0 1 'C# b5 b13']
Chord 2: 
[1 2 3 0 0 2 0 'C# #5 #11']
Notes 1:  {1, 5, 9, 7}
Notes 2:  {1, 5, 9, 7}
Chord 1: 
[1 2 1 0 0 0 1 'C# b5 b13']
Chord 2: 
[5 2 3 0 2 0 0 'F #5 9']
Notes 1:  {1, 5, 9, 7}
Notes 2:  {9, 5, 1, 7}
Chord 1: 
[1 2 1 0 0 0 1 'C# b5 b13']
Chord 2: 
[9 2 3 1 0 0 0 'A #5 7']
Notes 1:  {1, 5, 9, 7}
Notes 2:  {9, 5, 1, 7}
632
Chord 1: 
[1 2 1 0 0 0 2 'C# b5 13']
Chord 2: 
[7 1 1 1 0 0 0 'Gm b5 7']
Notes 1:  {1, 10, 5, 7}
Notes 2:  {1, 10, 5, 7}
Chord 1: 
[1 2 1 0 0 0 2 'C# b5 13']
Chord 2: 
[10 1 2 0 0 0 2 'Bbm 13']
Notes 1:  {1, 10, 5, 7}
Notes 2:  {1, 10, 5, 7}
633
634
Chord 1: 
[1 2 1 0 0 1 1 'C# b5 11 b13']
Chord 2: 
[

Chord 1: 
[1 2 1 1 1 0 2 'C# b5 7 b9 13']
Chord 2: 
[2 1 3 2 0 1 2 'Dm #5 maj7 11 13']
Notes 1:  {1, 2, 5, 7, 10, 11}
Notes 2:  {1, 2, 5, 7, 10, 11}
Chord 1: 
[1 2 1 1 1 0 2 'C# b5 7 b9 13']
Chord 2: 
[7 2 2 1 3 2 0 'G 7 #9 #11']
Notes 1:  {1, 2, 5, 7, 10, 11}
Notes 2:  {1, 2, 5, 7, 10, 11}
Chord 1: 
[1 2 1 1 1 0 2 'C# b5 7 b9 13']
Chord 2: 
[11 1 1 2 2 0 1 'Bm b5 maj7 9 b13']
Notes 1:  {1, 2, 5, 7, 10, 11}
Notes 2:  {1, 2, 5, 7, 10, 11}
Chord 1: 
[1 2 1 1 1 0 2 'C# b5 7 b9 13']
Chord 2: 
[11 1 3 2 2 2 0 'Bm #5 maj7 9 #11']
Notes 1:  {1, 2, 5, 7, 10, 11}
Notes 2:  {1, 2, 5, 7, 10, 11}
663
Chord 1: 
[1 2 1 1 1 1 0 'C# b5 7 b9 11']
Chord 2: 
[11 1 2 0 2 2 1 'Bm 9 #11 b13']
Notes 1:  {1, 2, 5, 6, 7, 11}
Notes 2:  {1, 2, 5, 6, 7, 11}
664
Chord 1: 
[1 2 1 1 1 1 1 'C# b5 7 b9 11 b13']
Chord 2: 
[2 2 2 2 3 1 2 'D maj7 #9 11 13']
Notes 1:  {1, 2, 5, 6, 7, 9, 11}
Notes 2:  {1, 2, 5, 6, 7, 9, 11}
Chord 1: 
[1 2 1 1 1 1 1 'C# b5 7 b9 11 b13']
Chord 2: 
[6 1 2 2 1 1 1 'F#m maj7 b9 11 b13']
Notes 1

Chord 1: 
[1 2 1 2 0 1 2 'C# b5 maj7 11 13']
Chord 2: 
[6 2 2 2 1 2 0 'F# maj7 b9 #11']
Notes 1:  {0, 1, 5, 6, 7, 10}
Notes 2:  {0, 1, 5, 6, 7, 10}
684
685
Chord 1: 
[1 2 1 2 1 0 1 'C# b5 maj7 b9 b13']
Chord 2: 
[1 2 3 2 1 2 0 'C# #5 maj7 b9 #11']
Notes 1:  {0, 1, 2, 5, 7, 9}
Notes 2:  {0, 1, 2, 5, 7, 9}
Chord 1: 
[1 2 1 2 1 0 1 'C# b5 maj7 b9 b13']
Chord 2: 
[9 2 3 1 3 1 0 'A #5 7 #9 11']
Notes 1:  {0, 1, 2, 5, 7, 9}
Notes 2:  {0, 1, 2, 5, 7, 9}
686
687
688
Chord 1: 
[1 2 1 2 1 1 1 'C# b5 maj7 b9 11 b13']
Chord 2: 
[6 1 2 2 1 2 1 'F#m maj7 b9 #11 b13']
Notes 1:  {0, 1, 2, 5, 6, 7, 9}
Notes 2:  {0, 1, 2, 5, 6, 7, 9}
Chord 1: 
[1 2 1 2 1 1 1 'C# b5 maj7 b9 11 b13']
Chord 2: 
[9 2 3 1 3 1 2 'A #5 7 #9 11 13']
Notes 1:  {0, 1, 2, 5, 6, 7, 9}
Notes 2:  {0, 1, 2, 5, 6, 7, 9}
689
Chord 1: 
[1 2 1 2 1 1 2 'C# b5 maj7 b9 11 13']
Chord 2: 
[6 2 2 2 1 2 1 'F# maj7 b9 #11 b13']
Notes 1:  {0, 1, 2, 5, 6, 7, 10}
Notes 2:  {0, 1, 2, 5, 6, 7, 10}
690
691
Chord 1: 
[1 2 1 2 2 0 1 'C# b5 maj7 9 b13']
C

Chord 1: 
[1 2 2 0 1 2 0 'C# b9 #11']
Chord 2: 
[5 1 3 0 2 0 2 'Fm #5 9 13']
Notes 1:  {1, 2, 5, 7, 8}
Notes 2:  {1, 2, 5, 7, 8}
718
719
Chord 1: 
[1 2 2 0 1 2 2 'C# b9 #11 13']
Chord 2: 
[2 1 1 2 0 1 1 'Dm b5 maj7 11 b13']
Notes 1:  {1, 2, 5, 7, 8, 10}
Notes 2:  {1, 2, 5, 7, 8, 10}
Chord 1: 
[1 2 2 0 1 2 2 'C# b9 #11 13']
Chord 2: 
[5 1 3 0 2 1 2 'Fm #5 9 11 13']
Notes 1:  {1, 2, 5, 7, 8, 10}
Notes 2:  {1, 2, 5, 7, 8, 10}
Chord 1: 
[1 2 2 0 1 2 2 'C# b9 #11 13']
Chord 2: 
[7 1 2 1 1 2 0 'Gm 7 b9 #11']
Notes 1:  {1, 2, 5, 7, 8, 10}
Notes 2:  {1, 2, 5, 7, 8, 10}
Chord 1: 
[1 2 2 0 1 2 2 'C# b9 #11 13']
Chord 2: 
[10 2 2 1 3 0 2 'Bb 7 #9 13']
Notes 1:  {1, 2, 5, 7, 8, 10}
Notes 2:  {1, 2, 5, 7, 8, 10}
720
Chord 1: 
[1 2 2 0 2 0 0 'C# 9']
Chord 2: 
[5 1 3 1 0 0 0 'Fm #5 7']
Notes 1:  {8, 1, 3, 5}
Notes 2:  {8, 1, 3, 5}
721
Chord 1: 
[1 2 2 0 2 0 1 'C# 9 b13']
Chord 2: 
[5 2 3 1 3 0 0 'F #5 7 #9']
Notes 1:  {1, 3, 5, 8, 9}
Notes 2:  {1, 3, 5, 8, 9}
Chord 1: 
[1 2 2 0 2 0 1 'C# 9 b13']
Chor

KeyboardInterrupt: 

In [ ]:
chord_data.to_csv(
    r'E:\Academics\Jupyter Notebooks\data\Chord Data\chords dataset.txt', 
    header='infer', 
    index=None, 
    sep=' ', 
    mode='a'
)

In [55]:
print(set(keys_of_chord([9, 1, 3, 0, 0, 0, 0])))
print(keys_of_chord([5, 2, 2, 0, 0, 0, 0]).sort)

{0, 9, 5}
<built-in method sort of numpy.ndarray object at 0x0000012C0CFE5A30>
